# Generate Prime Numbers
We will need some prime numbers. By using this function, we can generate a list of primes below the given $n$.

In [0]:
import numpy as np

def get_primes(n):
    """ Generates a list of prime number below the given n. """
    sieve = [True] * n
    for i in range(3, int(n**0.5) + 1,2):
        if sieve[i]:
            sieve[i*i::2*i] = [False] * int((n-i * i-1)/(2*i)+1)
    return [2] + [i for i in range(3,n,2) if sieve[i]]

# Generate Prime Numbers
First we need to calculate a $n = p \cdot q$ with both $p$ and $n$ being large prime numbers. This makes $n$ very hard to factorize. Here we first generate a list of prime numbers by using our function from above. We then pick random elements from the second half of that list to ensure we use sufficiently large primes. The $n$ we picked will be used throughout the whole process.

In [38]:
list_of_primes = get_primes(1000)

p = list_of_primes[np.random.randint(low = int(len(list_of_primes)/2), high = len(list_of_primes))]
q = list_of_primes[np.random.randint(low = int(len(list_of_primes)/2), high = len(list_of_primes))]

n = p * q

print(n)

287879


# Pick Secret s and Calculate v
A chooses a secret $s$ and calculates $v = s^2 mod\;n$. The $v$ will be public. However you can't easily compute $s$ if you only know $v$. Simply trying to find $s$ by brute force is therefore insufficient.

In [39]:
s = np.random.randint(low = 1000, high = 10000)

v = (s**2)%n
print(v)

55839


# Commitment
A now chooses a random integer $r$ to the Verifier V. P calculates $x = r^2 mod\;n$ and then sends $x$ to V.

In [40]:
r = np.random.randint(low = 1000, high = 10000)

x = (r**2)%n 
print(x)

203142


# Challenge
V now chooses a random bit $b \in \lbrace 0,1 \rbrace$. 

In [41]:
b = 1 if np.random.binomial(1, 0.5) == 1 else 0

print(b)

0


# Response
If V picked 0, A has to send $r\;mod\;n$. But if V picked 1, A has to send $(r \cdot s) mod \; n$. So V sends $(r \cdot s^b)mod\;n$. 

In [42]:
y = (r * (s**b))%n

print(y)

5411


# Verification
V checks if $y^2 \equiv x \cdot v\;\;(mod\;n)$.

In [43]:
if (y**2)%n == (x*(v**b))%n:
    print("valid")
else:
    print("something went wrong")

valid


# Fake Verification Attempt
In this scenario, E tries to verify without actually knowing the secret $s$. To do this, E has to guess the bit $b$, V will choose in advance. If E guesses the bit will be 0, the whole protocol would be passes without any changes, since knowing the secret $s$ is not necessary for this case. However, if V guesses $b=1$, E has to send $x = x \cdot \frac{1}{v}$. This will also be accepted since V checks if $y^2 \equiv x \cdot v^b\;(mod\;n)$ which is equivalent to $r^2 \equiv x \cdot v \cdot \frac{1}{v}\;(mod\;n)\Leftrightarrow r^2 \equiv x\;(mod\;n)$ which will obviously evaluate to True, since $x = r^2\;(mod\;n)$.


In [45]:
# initiate counter variables which will be used for output
valid_attempts = 0
invalid_attempts = 0

# we will go through the whole authentification process 1000 times
for i in range(100000):
    # E will have to choose a b in advance (which E can only do arbitrarily)
    fake_b = 1 if np.random.binomial(1, 0.5) == 1 else 0
    # choose a random bit as usual and calculate x
    r = np.random.randint(100, 1000)
    x = (r**2)%n
    if fake_b == 1:
        x = (1/v) * x

    
    # V now randomly chooses the bit b
    real_b = 1 if np.random.binomial(1, 0.5) == 1 else 0
    
    # E calculates y
    y = r%n

    # V checks if the authentification is valid
    if (y**2)%n == (round(x*(v**real_b),0))%n:
        valid_attempts += 1
    else:
        invalid_attempts += 1
        
        
        
print("Valid Attempts: ", valid_attempts)
print("Invalid Attempts: ", invalid_attempts)


Valid Attempts:  50157
Invalid Attempts:  49843


As seen above, the accepted and denied attempts should be equally distributed since E has a propability of $p = 0.5$ of picking the right bit.